In [23]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats

In [ ]:
def getAveragePer1000(filepath,n,filename):
    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    averages = [np.mean(csv_df.loc[i:i+n-1, "P"]) for i in range(0, csv_df.shape[0], n)]
    np.savetxt('../../similar_region_average1000/'+filename+'.csv', averages, delimiter = ',')
    return averages

In [ ]:
arrforDistribution=[]
for curDir, dirs, files in os.walk(top="../../similar_region_csvfiles/"):
    for file in files:
        if file.startswith("pos_prob_split_"):
            fileprefix=file[file.index("chr"):file.index(".")]
            filepath=os.path.join(curDir,file)
            arrforDistribution+=getAveragePer1000(filepath=filepath,n=1000,filename=fileprefix)

In [ ]:
#get mean of Distribution
mean_arr=np.mean(arrforDistribution)
#get standard deviation of Distribution
std_arr=np.std(arrforDistribution)

In [24]:
def compareWith0_01Pvalue(mean_arr,std_arr,filename,filepath):
    arr=[]
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    for i in range(len(csv_matrix)):
        mid_res=stats.norm.cdf(csv_matrix[i], mean_arr, std_arr)
        if mid_res>0.95:
            arr.append(1)
        else:
            arr.append(0)
    np.savetxt("../../similar_region_95_compare/"+filename+".csv", arr, delimiter = ',')
    return filename

In [26]:
for curDir, dirs, files in os.walk(top="../../similar_region_average1000/"):
    for file in files:            
        filename=file[:file.index(".")]
        prefix=filename.split("_")[0]
        filepath=os.path.join(curDir,file)
        #print(filepath)
        compareWith0_01Pvalue(mean_arr=mean_arr,std_arr=std_arr,filename=filename,filepath=filepath)

In [27]:
def generatedfandcsv(columns,dic,filename):

    row_names=list(dic.keys())
    a=[]
    bs=[]
    #cs=[]
    for x in row_names:
        a.append(x)
        bs.append(dic[x])
        #cs.append(dic[x][1])
    dictforDF=dict()
    dictforDF[columns[0]]=a
    dictforDF[columns[1]]=bs
    #dictforDF[columns[2]]=cs

    df_index=pd.DataFrame(dictforDF)
    df_index.to_csv("../../similar_region_95_start_length/"+filename+"_start_lenth.csv")
    return df_index


In [28]:
def checkContinuous1(arr):
    res=dict()
    i=0
    while i<len(arr)-1:

        if arr[i]==1:

            count=1
            for j in range(i+1,len(arr),1):
                if arr[j]==1:
                    count+=1
                    
                else:
                    
                    break

            res[i]=count
            i+=count

        else:
            i+=1
            continue
    return res
            


In [29]:
def getstartAndLength(filepath,filename):
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    res=checkContinuous1(csv_matrix)
    df=generatedfandcsv(columns=["start","length"],dic=res,filename=filename)
    return df

In [30]:
for curDir, dirs, files in os.walk(top="../../similar_region_95_compare/"):
    for file in files:
        filename=file[:file.index(".")]
        filepath=os.path.join(curDir,file)
        getstartAndLength(filepath=filepath,filename=filename)

In [31]:

def countNumofLength(dic,filepath):
    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr=np.array(csv_df['length'])
    #print(dic)
    for i in range(len(arr)):
        #print("before iteration",dic)
        if arr[i] in dic:
            #print("in iteration not",dic)
            dic[arr[i]]+=1
        else:
            dic[arr[i]]=1
            #print("in iteration",dic)
    #print(dic)
    return dic

In [32]:

mydic=dict()
mhcdic=dict()
for curDir, dirs, files in os.walk(top="../../similar_region_95_start_length/"):
    for file in files:
            filename=file[:file.index(".")]
            if filename!="mhc_start_lenth":                  
                filepath=os.path.join(curDir,file)
                countNumofLength(mydic,filepath=filepath)
            else:
                 filepath_mhc=os.path.join(curDir,file)
                 countNumofLength(mhcdic,filepath=filepath_mhc)

In [33]:
mhcdic

{}

In [34]:
mydic

{1: 318,
 2: 99,
 4: 29,
 11: 3,
 3: 61,
 5: 12,
 63: 1,
 15: 1,
 6: 8,
 100: 1,
 56: 1,
 7: 7,
 9: 1,
 13: 2,
 45: 1,
 8: 6,
 23: 1,
 33: 1,
 31: 1,
 79: 1,
 16: 1,
 59: 1,
 20: 1}

In [35]:
arr1=[]

for k2,arr in mydic.items():
    for i in range(arr):
        arr1.append(k2)

for k2,arr in mhcdic.items():
    for i in range(arr):
        arr1.append(k2)

In [37]:
mean_1=np.mean(arr1)
dev1=np.std(arr1)
print("mean",mean_1)
print("dev",dev1)
mid_res=stats.norm.ppf(0.9999999999,mean_1,dev1)
mid_res

mean 2.917562724014337
dev 7.49984562389563


50.62663735771988

In [38]:

def getAllPositions(filepath):
    pos=[]
    #vcf_in=vcf(filepath)
    vcf_file = pysam.VariantFile(filepath)
    for record in vcf_file:

        pos.append(record.pos)
    pos=sorted(list(set(pos)))
    return pos

In [39]:
def findPosition(filepath,start,length):
    arr=[]
    allpos=getAllPositions(filepath=filepath)
    arr.append(allpos[start*1000])
    if (start+length)*1000-1<len(allpos):
        end=allpos[(start+length)*1000-1]
    else:
        end=allpos[-1]
    arr.append(end)
    return arr



In [40]:

def GetPosition(filepath,res,threshold):
    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]
    filename=(filepath.split("/")[-1]).split("_start_lenth.csv")[0]
    findfoldername="split_"+filename.split("_")[0]
    findfoldername_filename=filename.split("_")[1].split(".")[0]
    filepath="../../"+findfoldername+"/"+findfoldername_filename+".vcf.gz"
    for k,v in mydic.items():
        if v>=threshold:
            print("filepath",filepath)
            midarr.append(findPosition(filepath=filepath,start=k,length=v))
            
        else:
            continue
    if midarr!=[]:
        res[findfoldername+"_"+findfoldername_filename]=midarr
    return res

In [41]:
def getPositionforMHC(res):
    mhc_start_length_path="/data2/wangxuedong/mhc_test_data/similar_region_95_start_length/mhc_start_lenth.csv"
    csv_data = pd.read_csv(mhc_start_length_path, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]
    for k,v in mydic.items():
        if v>=51:
            midarr.append(findPosition(filepath="../../mhcdataset/1kgp.29720000-33130000.vcf",start=k,length=v))            
        else:
            continue
    res["MHC"]=midarr
    return res
            



In [42]:
res=dict()
mhcres=dict()
for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/similar_region_95_start_length/"):
    for file in files:
            filename=file[:file.index(".")]
            if filename!="mhc_start_lenth":
                filepath=os.path.join(curDir,file)
                GetPosition(filepath=filepath,res=res,threshold=51)
            else:
                getPositionforMHC(res=mhcres)

filepath ../../split_chr13/xaa.vcf.gz


[E::idx_find_and_load] Could not retrieve index file for '../../split_chr13/xaa.vcf.gz'


filepath ../../split_chr1/xba.vcf.gz


[E::idx_find_and_load] Could not retrieve index file for '../../split_chr1/xba.vcf.gz'


filepath ../../split_chr1/xbb.vcf.gz


[E::idx_find_and_load] Could not retrieve index file for '../../split_chr1/xbb.vcf.gz'


filepath ../../split_chr2/xav.vcf.gz


[E::idx_find_and_load] Could not retrieve index file for '../../split_chr2/xav.vcf.gz'


filepath ../../split_chr3/xau.vcf.gz


[E::idx_find_and_load] Could not retrieve index file for '../../split_chr3/xau.vcf.gz'


In [43]:
res

{'split_chr13_xaa': [[16000187, 18181463]],
 'split_chr1_xba': [[121579777, 123992934]],
 'split_chr1_xbb': [[123992935, 124938490]],
 'split_chr2_xav': [[91720904, 94530414]],
 'split_chr3_xau': [[90362505, 91477491]]}

In [44]:
mhcres

{'MHC': []}